## Match searches with bookings

- For every search in the searches file, find out whether the search ended up in a booking or not (using the info in the bookings file). For instance, search and booking origin and destination should match. 

- For the bookings file, origin and destination are the columns dep_port and arr_port, respectively. 

- Generate a CSV file with the search data, and an additional field, containing 1 if the search ended up in a booking, and 0 otherwise.



## 2) Prepare the data for processing

### Booking

#### We didnt check for duplicates so far... What if the file is has duplicated lines?

http://stackoverflow.com/questions/24251219/pandas-read-csv-low-memory-and-dtype-options

In [35]:
import pandas as pd

bookings_file = '../data/Challenge/bookings.csv.bz2'
searches_file = '../data/Challenge/searches.csv.bz2'

bookings_sample = pd.read_csv(bookings_file, sep='^', nrows=100000)

In [36]:
pd.options.display.max_columns = None
bookings_sample.head(5)

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,dep_city,dep_ctry,arr_port,arr_city,arr_ctry,lst_port,lst_city,lst_ctry,brd_port,brd_city,brd_ctry,off_port,off_city,off_ctry,mkt_port,mkt_city,mkt_ctry,intl,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,2013-03-05 00:00:00,1A,DE,a68dd7ae953c8acfb187a1af2dcbe123,1a11ae49fcbf545fd2afc1a24d88d2b7,ea65900e72d71f4626378e2ebd298267,2013-02-22 00:00:00,1708,0,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHRZRH,LONZRH,CHGB,1,LHRZRH,VI,T,Y,2013-03-07 08:50:00,2013-03-07 11:33:37,-1,2013,3,NULL
1,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,SAL,SAL,SV,CLT,CLT,US,CLTSAL,CLTSAL,SVUS,1,SALATLCLT,NV,L,Y,2013-04-12 13:04:00,2013-04-12 22:05:40,1,2013,3,NULL
2,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,CLTSAL,CLTSAL,SVUS,1,CLTATLSAL,NV,U,Y,2013-07-15 07:00:00,2013-07-15 11:34:51,1,2013,3,NULL
3,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,AKL,AKL,NZ,SVO,MOW,RU,AKLSVO,AKLMOW,NZRU,1,AKLHKGSVO,XK,G,Y,2013-04-24 23:59:00,2013-04-25 16:06:31,1,2013,3,SYDA82546
4,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,AKLSVO,AKLMOW,NZRU,1,SVOHKGAKL,XK,G,Y,2013-05-14 20:15:00,2013-05-16 10:44:50,1,2013,3,SYDA82546


### * Duplicates

In [37]:
%%time
from datetime import datetime

bookings = pd.read_csv(bookings_file, sep='^', chunksize=1e6)
partial_results = pd.DataFrame()

for chunk_num, chunk in enumerate(bookings):
    print('Start with chunk %d in bookings at %s' % (chunk_num, datetime.now()))
    
    partial_results = partial_results.append(chunk)
    partial_results.drop_duplicates(inplace=True)
    print('After chunk %d partial_results has shape %s' % (chunk_num, partial_results.shape))

print('Start with final dedup at %s' % datetime.now())
    
bookings_nodups = partial_results
bookings_nodups.shape

Start with chunk 0 in bookings at 2018-06-02 09:44:17.631022
After chunk 0 partial_results has shape (1000000, 38)
Start with chunk 1 in bookings at 2018-06-02 09:44:41.831715
After chunk 1 partial_results has shape (1000000, 38)
Start with chunk 2 in bookings at 2018-06-02 09:45:09.861275
After chunk 2 partial_results has shape (1000000, 38)
Start with chunk 3 in bookings at 2018-06-02 09:45:35.011434
After chunk 3 partial_results has shape (1000000, 38)
Start with chunk 4 in bookings at 2018-06-02 09:46:04.584135
After chunk 4 partial_results has shape (1000000, 38)


<string>:2: DtypeWarning: Columns (7,8,27) have mixed types. Specify dtype option on import or set low_memory=False.


Start with chunk 5 in bookings at 2018-06-02 09:46:29.587203
After chunk 5 partial_results has shape (1016377, 38)
Start with chunk 6 in bookings at 2018-06-02 09:46:56.663626
After chunk 6 partial_results has shape (1016377, 38)
Start with chunk 7 in bookings at 2018-06-02 09:47:29.335532
After chunk 7 partial_results has shape (1016377, 38)
Start with chunk 8 in bookings at 2018-06-02 09:47:54.779167
After chunk 8 partial_results has shape (1016377, 38)
Start with chunk 9 in bookings at 2018-06-02 09:48:21.952903
After chunk 9 partial_results has shape (1016377, 38)
Start with chunk 10 in bookings at 2018-06-02 09:48:32.276819
After chunk 10 partial_results has shape (1016377, 38)
Start with final dedup at 2018-06-02 09:48:37.986269
CPU times: user 4min 50s, sys: 4.31 s, total: 4min 54s
Wall time: 4min 39s


#### We have seen that we have white space in some columns....

In [38]:
bookings_sample.columns = bookings_sample.columns.str.strip()

###  Search

In [39]:
searches_sample = pd.read_csv(searches_file, sep='^', nrows=100000)

searches_sample.head()

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,Seg1Arrival,Seg1Date,Seg1Carrier,Seg1BookingCode,Seg2Departure,Seg2Arrival,Seg2Date,Seg2Carrier,Seg2BookingCode,Seg3Departure,Seg3Arrival,Seg3Date,Seg3Carrier,Seg3BookingCode,Seg4Departure,Seg4Arrival,Seg4Date,Seg4Carrier,Seg4BookingCode,Seg5Departure,Seg5Arrival,Seg5Date,Seg5Carrier,Seg5BookingCode,Seg6Departure,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1,2,TXL,AUH,2013-01-26,D2,NaN,AUH,TXL,2013-02-02,D2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0,1,ATH,MIL,2013-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV
2,2013-01-01,18:04:49,MPT,3561a60621de06ab1badc8ca55699ef3,US,ICT,SFO,1,2,ICT,SFO,2013-08-02,NaN,NaN,SFO,ICT,2013-08-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,NYC
3,2013-01-01,17:42:40,FXP,1864e5e8013d9414150e91d26b6a558b,SE,RNB,ARN,0,1,RNB,ARN,2013-01-02,DU,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,STO
4,2013-01-01,17:48:29,MPT,1ec336348f44207d2e0027dc3a68c118,NO,OSL,MAD,1,2,OSL,MAD,2013-03-22,NaN,NaN,MAD,OSL,2013-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,OSL


## 3) Make action plan

Generate a CSV file with the search data, and an additional field, containing 1 if the search ended up in a booking, and 0 otherwise.

1) remove duplicates

2) parse dates from string to datetime

3) remove whitespaces

        a) from colum names
        
        b) from content
        
4) remove NaN

5) define the model

    if there is one booking for a given O&D done at the same day as the search (for a given O&D), ALL searches of the day (for a given 0&D) might have resulted from the same source and will be set with 1.
    This is regardless of the boarding time of the plane... So if I was looking for plane for the first 4 days of December for a given O&D all those searches would be set to 1 not just the one correspoding to the correct boarding time

        match
        Search : [search_date, O&D] 
        Booking: [creation_date, O&D]

6) execute the model

        a) Group by bookings on [cre_date, O&D] so that we dont have duplicates 
        (and we can have number of bookings for the day)
        or we can just drop the duplicates
        b) search left join bookings adding "Booked" column
        c) test if the merge was done right
        d) fill NaN of "booked" column with 0
        e) pull all values of booked column >1 to 1

more complex... get number of segments from searches
match search_date, and then split all O&D of all segments, and match the date of first flight of each segment (seg1Date, seg2Date)
with boarding time and O&D and act_date of booking

What do we have?

In [40]:
bookings_sample.head()

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,dep_city,dep_ctry,arr_port,arr_city,arr_ctry,lst_port,lst_city,lst_ctry,brd_port,brd_city,brd_ctry,off_port,off_city,off_ctry,mkt_port,mkt_city,mkt_ctry,intl,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,2013-03-05 00:00:00,1A,DE,a68dd7ae953c8acfb187a1af2dcbe123,1a11ae49fcbf545fd2afc1a24d88d2b7,ea65900e72d71f4626378e2ebd298267,2013-02-22 00:00:00,1708,0,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHRZRH,LONZRH,CHGB,1,LHRZRH,VI,T,Y,2013-03-07 08:50:00,2013-03-07 11:33:37,-1,2013,3,NULL
1,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,SAL,SAL,SV,CLT,CLT,US,CLTSAL,CLTSAL,SVUS,1,SALATLCLT,NV,L,Y,2013-04-12 13:04:00,2013-04-12 22:05:40,1,2013,3,NULL
2,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,CLTSAL,CLTSAL,SVUS,1,CLTATLSAL,NV,U,Y,2013-07-15 07:00:00,2013-07-15 11:34:51,1,2013,3,NULL
3,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,AKL,AKL,NZ,SVO,MOW,RU,AKLSVO,AKLMOW,NZRU,1,AKLHKGSVO,XK,G,Y,2013-04-24 23:59:00,2013-04-25 16:06:31,1,2013,3,SYDA82546
4,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,AKLSVO,AKLMOW,NZRU,1,SVOHKGAKL,XK,G,Y,2013-05-14 20:15:00,2013-05-16 10:44:50,1,2013,3,SYDA82546


In [41]:
non_cancelled = bookings_sample[bookings_sample['pax'] > 0]
non_cancelled = non_cancelled[['cre_date', 'dep_port', 'arr_port']]

In [42]:
non_cancelled['cre_date'] = non_cancelled['cre_date'].str[:10]

In [43]:
non_cancelled.head()

,cre_date,dep_port,arr_port
1,2013-03-26,SAL,CLT
2,2013-03-26,SAL,CLT
3,2013-03-26,AKL,SVO
4,2013-03-26,AKL,SVO
5,2013-03-20,DEN,LGA


In [44]:
non_cancelled.shape

(68654, 3)

In [45]:
non_cancelled = non_cancelled.drop_duplicates()

In [46]:
bookings_sample.columns

Index(['act_date', 'source', 'pos_ctry', 'pos_iata', 'pos_oid', 'rloc',
       'cre_date', 'duration', 'distance', 'dep_port', 'dep_city', 'dep_ctry',
       'arr_port', 'arr_city', 'arr_ctry', 'lst_port', 'lst_city', 'lst_ctry',
       'brd_port', 'brd_city', 'brd_ctry', 'off_port', 'off_city', 'off_ctry',
       'mkt_port', 'mkt_city', 'mkt_ctry', 'intl', 'route', 'carrier',
       'bkg_class', 'cab_class', 'brd_time', 'off_time', 'pax', 'year',
       'month', 'oid'],
      dtype='object')

In [47]:
%%time
from datetime import datetime

bookings = pd.read_csv(bookings_file, 
                       sep='^', 
                       chunksize=1000000, 
                       usecols=['cre_date           ', 'dep_port', 'arr_port', 'pax'])
partial_results = []

for chunk_num, chunk in enumerate(bookings):
    
    print('Start with chunk %d in bookings at %s' % (chunk_num, datetime.now()))
    
    chunk.columns = chunk.columns.str.strip()
    non_cancelled = chunk[chunk['pax'] > 0]
    non_cancelled['cre_date'] = non_cancelled['cre_date'].str[:10]
    non_cancelled = non_cancelled[['cre_date', 'dep_port', 'arr_port']]
    
    partial_results.append(non_cancelled.drop_duplicates())
    
bookings_nodups = pd.concat(partial_results).drop_duplicates()

Start with chunk 0 in bookings at 2018-06-02 09:48:51.012950


/home/dani/anaconda3/envs/master-p3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


Start with chunk 1 in bookings at 2018-06-02 09:49:04.536842
Start with chunk 2 in bookings at 2018-06-02 09:49:17.265741
Start with chunk 3 in bookings at 2018-06-02 09:49:31.050072
Start with chunk 4 in bookings at 2018-06-02 09:49:44.274210
Start with chunk 5 in bookings at 2018-06-02 09:49:58.490810
Start with chunk 6 in bookings at 2018-06-02 09:50:11.814466
Start with chunk 7 in bookings at 2018-06-02 09:50:25.585009
Start with chunk 8 in bookings at 2018-06-02 09:50:38.375205
Start with chunk 9 in bookings at 2018-06-02 09:50:50.359288
Start with chunk 10 in bookings at 2018-06-02 09:50:50.845601
CPU times: user 2min 13s, sys: 172 ms, total: 2min 14s
Wall time: 2min 12s


In [49]:
bookings_nodups.shape

(315574, 3)

In [50]:
searches_sample.head(5)

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,Seg1Arrival,Seg1Date,Seg1Carrier,Seg1BookingCode,Seg2Departure,Seg2Arrival,Seg2Date,Seg2Carrier,Seg2BookingCode,Seg3Departure,Seg3Arrival,Seg3Date,Seg3Carrier,Seg3BookingCode,Seg4Departure,Seg4Arrival,Seg4Date,Seg4Carrier,Seg4BookingCode,Seg5Departure,Seg5Arrival,Seg5Date,Seg5Carrier,Seg5BookingCode,Seg6Departure,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1,2,TXL,AUH,2013-01-26,D2,NaN,AUH,TXL,2013-02-02,D2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0,1,ATH,MIL,2013-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV
2,2013-01-01,18:04:49,MPT,3561a60621de06ab1badc8ca55699ef3,US,ICT,SFO,1,2,ICT,SFO,2013-08-02,NaN,NaN,SFO,ICT,2013-08-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,NYC
3,2013-01-01,17:42:40,FXP,1864e5e8013d9414150e91d26b6a558b,SE,RNB,ARN,0,1,RNB,ARN,2013-01-02,DU,W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,STO
4,2013-01-01,17:48:29,MPT,1ec336348f44207d2e0027dc3a68c118,NO,OSL,MAD,1,2,OSL,MAD,2013-03-22,NaN,NaN,MAD,OSL,2013-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,OSL


In [51]:
%%time

merged = searches_sample.merge(bookings_nodups, 
                               left_on=['Origin', 'Destination', 'Date'], 
                               right_on=['dep_port', 'arr_port', 'cre_date'])

CPU times: user 244 ms, sys: 8 ms, total: 252 ms
Wall time: 250 ms


In [52]:
merged

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,Seg1Arrival,Seg1Date,Seg1Carrier,Seg1BookingCode,Seg2Departure,Seg2Arrival,Seg2Date,Seg2Carrier,Seg2BookingCode,Seg3Departure,Seg3Arrival,Seg3Date,Seg3Carrier,Seg3BookingCode,Seg4Departure,Seg4Arrival,Seg4Date,Seg4Carrier,Seg4BookingCode,Seg5Departure,Seg5Arrival,Seg5Date,Seg5Carrier,Seg5BookingCode,Seg6Departure,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,cre_date,dep_port,arr_port


Why is it that the join has returned empty????


The data in two of the columns also has embedded whitespace, just like the column names.

In [55]:
bookings_nodups['arr_port'].str.len().head()

1     8
3     8
5     8
7     8
19    8
Name: arr_port, dtype: int64

In [56]:
%%time
from datetime import datetime

bookings = pd.read_csv(bookings_file, 
                       sep='^', 
                       chunksize=1000000, 
                       usecols=['cre_date           ', 'dep_port', 'arr_port', 'pax'])
partial_results = []

for chunk_num, chunk in enumerate(bookings):
    
    print('Start with chunk %d in bookings at %s' % (chunk_num, datetime.now()))
    
    chunk.columns = chunk.columns.str.strip()
    chunk['dep_port'] = chunk['dep_port'].str.strip()
    chunk['arr_port'] = chunk['arr_port'].str.strip()
    
    non_cancelled = chunk[chunk['pax'] > 0]
    non_cancelled['cre_date'] = non_cancelled['cre_date'].str[:10]
    non_cancelled = non_cancelled[['cre_date', 'dep_port', 'arr_port']]
    
    partial_results.append(non_cancelled.drop_duplicates())
    
bookings_nodups = pd.concat(partial_results).drop_duplicates()

Start with chunk 0 in bookings at 2018-06-02 09:51:07.301973


/home/dani/anaconda3/envs/master-p3/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Start with chunk 1 in bookings at 2018-06-02 09:51:23.755771
Start with chunk 2 in bookings at 2018-06-02 09:51:41.166877
Start with chunk 3 in bookings at 2018-06-02 09:51:59.952244
Start with chunk 4 in bookings at 2018-06-02 09:52:16.776048
Start with chunk 5 in bookings at 2018-06-02 09:52:34.945698
Start with chunk 6 in bookings at 2018-06-02 09:52:52.068158
Start with chunk 7 in bookings at 2018-06-02 09:53:08.871198
Start with chunk 8 in bookings at 2018-06-02 09:53:26.514054
Start with chunk 9 in bookings at 2018-06-02 09:53:44.511617
Start with chunk 10 in bookings at 2018-06-02 09:53:46.245310
CPU times: user 2min 56s, sys: 256 ms, total: 2min 56s
Wall time: 2min 55s


In [57]:
(bookings_nodups['dep_port'].str.len() != 3.0).sum()

0

In [58]:
%%time

merged = searches_sample.merge(bookings_nodups, 
                               left_on=['Origin', 'Destination', 'Date'], 
                               right_on=['dep_port', 'arr_port', 'cre_date'],
                               how='left')

CPU times: user 380 ms, sys: 0 ns, total: 380 ms
Wall time: 374 ms


In [59]:
merged.shape

(100000, 48)

In [60]:
searches_sample.shape

(100000, 45)

In [61]:
merged['Booking'] = merged['cre_date'].notnull().astype(int)

In [62]:
merged = merged.drop(['cre_date', 'dep_port', 'arr_port'], axis=1)

Now it seems we got everything we need and not more: we haven't caused duplication or loss of records. We are ready to extend the approach to the whole dataset.

The problem text specifies that we need to write a csv out. Therefore, we don't even need to build the whole DataFrame in memory, we just need to write the partial results out as we get them. We can use `mode='a'` in order to append each chunk to the same file- we just need to be careful to write the header only once.

In [65]:
%%time

searches = pd.read_csv(searches_file, sep='^', chunksize=2e6)

for chunk_num, chunk in enumerate(searches):
    
    print('Start with chunk %d in searches at %s' % (chunk_num, datetime.now()))

    merged = chunk.merge(bookings_nodups, 
                         left_on=['Origin', 'Destination', 'Date'], 
                         right_on=['dep_port', 'arr_port', 'cre_date'],
                         how='left')
    
    merged['Booking'] = merged['cre_date'].notnull().astype(int)
    merged = merged.drop(['cre_date', 'dep_port', 'arr_port'], axis=1)
    
    # We don't want to build a huge DataFrame in memory
    # partial_results.append(merged)
    
    merged.to_csv('annotated_searches.csv.bz2' , 
                  compression='bz2', 
                  sep='^', 
                  mode='a',
                  header=chunk_num==0)

<string>:2: DtypeWarning: Columns (40,41,42,44) have mixed types. Specify dtype option on import or set low_memory=False.


Start with chunk 0 in searches at 2018-06-02 09:54:40.613550
Start with chunk 1 in searches at 2018-06-02 09:56:14.654398
Start with chunk 2 in searches at 2018-06-02 09:57:42.127803
Start with chunk 3 in searches at 2018-06-02 09:59:15.424818
Start with chunk 4 in searches at 2018-06-02 10:00:38.196617
Start with chunk 5 in searches at 2018-06-02 10:02:04.561314
Start with chunk 6 in searches at 2018-06-02 10:03:30.744523
Start with chunk 7 in searches at 2018-06-02 10:05:02.908006
Start with chunk 8 in searches at 2018-06-02 10:06:41.143024
Start with chunk 9 in searches at 2018-06-02 10:08:16.247435


<string>:2: DtypeWarning: Columns (44) have mixed types. Specify dtype option on import or set low_memory=False.


Start with chunk 10 in searches at 2018-06-02 10:09:30.319293
CPU times: user 15min 26s, sys: 7.54 s, total: 15min 33s
Wall time: 15min 29s


In [66]:
%%time

searches = pd.read_csv(searches_file, sep='^', chunksize=2e5)

for chunk_num, chunk in enumerate(searches):
    
    print('Start with chunk %d in searches at %s' % (chunk_num, datetime.now()))

    merged = chunk.merge(bookings_nodups, 
                         left_on=['Origin', 'Destination', 'Date'], 
                         right_on=['dep_port', 'arr_port', 'cre_date'],
                         how='left')
    
    merged['Booking'] = merged['cre_date'].notnull().astype(int)
    merged = merged.drop(['cre_date', 'dep_port', 'arr_port'], axis=1)
    
    # We don't want to build a huge DataFrame in memory
    # partial_results.append(merged)
    
    merged.to_csv('annotated_searches.csv' , 
                  compression='bz2', 
                  sep='^', 
                  mode='a',
                  header=not bool(chunk_num))

Start with chunk 0 in searches at 2018-06-02 12:03:29.175387


<string>:2: DtypeWarning: Columns (44) have mixed types. Specify dtype option on import or set low_memory=False.


Start with chunk 1 in searches at 2018-06-02 12:03:39.142704
Start with chunk 2 in searches at 2018-06-02 12:03:48.305885
Start with chunk 3 in searches at 2018-06-02 12:03:57.464726
Start with chunk 4 in searches at 2018-06-02 12:04:05.774574
Start with chunk 5 in searches at 2018-06-02 12:04:14.876704
Start with chunk 6 in searches at 2018-06-02 12:04:23.691213


<string>:2: DtypeWarning: Columns (40,41,42,44) have mixed types. Specify dtype option on import or set low_memory=False.


Start with chunk 7 in searches at 2018-06-02 12:04:32.113808
Start with chunk 8 in searches at 2018-06-02 12:04:41.175579
Start with chunk 9 in searches at 2018-06-02 12:04:50.125845
Start with chunk 10 in searches at 2018-06-02 12:04:59.992335
Start with chunk 11 in searches at 2018-06-02 12:05:09.150805
Start with chunk 12 in searches at 2018-06-02 12:05:18.989988
Start with chunk 13 in searches at 2018-06-02 12:05:27.994860
Start with chunk 14 in searches at 2018-06-02 12:05:37.993865
Start with chunk 15 in searches at 2018-06-02 12:05:46.965924
Start with chunk 16 in searches at 2018-06-02 12:05:55.540427
Start with chunk 17 in searches at 2018-06-02 12:06:04.793278
Start with chunk 18 in searches at 2018-06-02 12:06:13.327391
Start with chunk 19 in searches at 2018-06-02 12:06:21.676781
Start with chunk 20 in searches at 2018-06-02 12:06:30.368936
Start with chunk 21 in searches at 2018-06-02 12:06:40.034564
Start with chunk 22 in searches at 2018-06-02 12:06:48.156121
Start with 

<string>:2: DtypeWarning: Columns (40,41,42) have mixed types. Specify dtype option on import or set low_memory=False.


Start with chunk 43 in searches at 2018-06-02 12:10:18.254094
Start with chunk 44 in searches at 2018-06-02 12:10:26.720764
Start with chunk 45 in searches at 2018-06-02 12:10:35.264006
Start with chunk 46 in searches at 2018-06-02 12:10:44.136494
Start with chunk 47 in searches at 2018-06-02 12:10:52.248607
Start with chunk 48 in searches at 2018-06-02 12:11:01.509455
Start with chunk 49 in searches at 2018-06-02 12:11:11.762343
Start with chunk 50 in searches at 2018-06-02 12:11:24.015794
Start with chunk 51 in searches at 2018-06-02 12:11:34.717290
Start with chunk 52 in searches at 2018-06-02 12:11:43.973052
Start with chunk 53 in searches at 2018-06-02 12:11:53.124326
Start with chunk 54 in searches at 2018-06-02 12:12:03.037047
Start with chunk 55 in searches at 2018-06-02 12:12:12.717070
Start with chunk 56 in searches at 2018-06-02 12:12:21.798025
Start with chunk 57 in searches at 2018-06-02 12:12:30.479299
Start with chunk 58 in searches at 2018-06-02 12:12:41.739479
Start wi